## Primeramente importamos las librerias necesarias

In [9]:
# Librerias y definicion
import sympy as sp # Libreria de simbolos
from sympy import oo # Infinito
import numpy as np # Libreria numerica
from itertools import product # Producto cartesiano
from scipy.linalg import eig # Eigenvalores y eigenvectores
from sympy import diff # Derivadas
import time as time # Tiempo
import matplotlib.pyplot as plt # Graficas
from sympy.plotting import plot # Graficas
%matplotlib inline 
from IPython.display import Math # Mostrar ecuaciones
from IPython.display import Math


Luego definimos las funciones:

In [10]:

sp.init_printing() # Mostrar ecuaciones

r, r1, r2, zeta, zeta1, zeta2 = sp.symbols("r, r1, r2, zeta, zeta1, zeta2") # Definicion de simbolos
n = sp.Symbol('n',integer=True) # Definicion de simbolos

# Definicion de la funciones de estado con l=0

def STO(zeta, n, r=r): # Definicion de la funcion de estado
    return (2*zeta)**n*(2*zeta/sp.factorial(2*n))**(1/2)*r**(n-1)*sp.exp(-zeta*r)

# definicion del valor esperado del operador hamiltoniano sin considerar la repulcion entre electrones

def S_int(f1, f2):# Definicion de la integral de solapamiento
    return sp.integrate(f1*f2*r*r ,(r, 0, +oo))

def H_int(f1, f2, Z): # Definicion de la integral de hamiltoniano
    return sp.integrate(f1*(-((1/2)*(1/r)*diff(diff(r*f2, r), r))-((Z/r)*f2))*r*r, (r,0,+oo))

def H_matrix(fs, Z): # Definicion de la matriz de hamiltoniano
    H = np.zeros((len(fs),len(fs)))
    for i in range(len(fs)):
        for j in range(len(fs)):
            H[i, j] =  H_int(fs[i], fs[j], Z)
    return H    

# definicion del valor esperado del operador solapamiento
def S_matrix(fs): 
    S = np.zeros((len(fs),len(fs)))
    for i in range(len(fs)):
        for j in range(len(fs)):
            S[i, j] =  S_int(fs[i], fs[j])

    return S 

# definicion del valor esperado del operador hamiltoniano considerando la repulcion entre electrones
def Repulsion_electron(zetas): # Definicion de la integral de repulsion electronica
    f1=STO(zetas[0][0], zetas[0][1], r1)
    f2=STO(zetas[1][0], zetas[1][1], r1)
    f3=STO(zetas[2][0], zetas[2][1], r2)
    f4=STO(zetas[3][0], zetas[3][1], r2)
    fs = [f1, f2, f3, f4]
    B = (1/r1)*sp.integrate(f3*f4*r2*r2 ,(r2, 0, r1)) + sp.integrate((1/r2)*f3*f4*r2*r2 ,(r2, r1, +oo))
    A = sp.integrate(f1*f2*r1*r1*B ,(r1, 0, +oo))
    return A

def P_matrix(Co): # Definicion de la matriz de densidad
    P = np.zeros([Co.shape[0], Co.shape[0]])

    for t in range(Co.shape[0]):
        for u in range(Co.shape[0]):
            P[t][u] = 2* Co[t][0]*Co[u][0] + Co[t][1]*Co[u][1]
    return P 

# Energia empleando el metodo variacional 
def R_matrix(zetas):
    R = np.zeros((len(zetas), len(zetas), len(zetas), len(zetas)))

    rs = list(product(range(len(zetas)),repeat=2))
    tu = list(product(range(len(zetas)),repeat=2))

    for r, s in rs:
        for t, u in tu:
            R[r,s,t,u] = Repulsion_electron((zetas[r], zetas[s], zetas[t], zetas[u]))
    return R

def G_matrix(zetas, Co, R):

    G = np.zeros((Co.shape[0], Co.shape[0]))

    P = P_matrix(Co)
    
    rs = list(product(range( Co.shape[0]),repeat=2))
    tu = list(product(range( Co.shape[0]),repeat=2))

    for r, s in rs:
        g = 0
        for t, u in tu:
            int1 = R[r, s, t, u]
            int2 = R[r, u, t, s]
#             print('({0}{1}|{2}{3}): {4}'.format(r, s, t, u, int1))
            g+= P[t, u] * (int1 - 0.5 * int2)
        G[r, s] = g
    return G

def F_matrix(fs, Z, zetas, Co, R):
    return H_matrix(fs, Z) + G_matrix(zetas, Co, R)

# Ecuaciones de Hartree Fock

def secular_eqn(F, S):
    ei, C = eig(F, S)
    
    idx = ei.argsort()[::1]   
    ei = ei[idx]
    C = C[:,idx]
    
    Co = np.zeros((C.shape[0],C.shape[0]))
    inte = np.matmul(np.matmul(C.T, S), C)
    for i in range(C.shape[0]):
        for j in range(C.shape[0]):
            Co[j][i]=C[j][i]/np.sqrt(inte[i][i])

    return ei, Co

# Energia del atomo

def get_E0(e, P, H):
    
    E0 = 0
    for i in range(int(e.shape[0]/2)):
        E0 += e[i].real 
    E0 = E0 + 0.5*(P*H).sum()
    return E0


# Proceso iterativo para determinar la energia del atomo 


In [11]:
# Proceso iterativo para determinar la energia del atomo 

zetas = [[4.61679, 1], [2.46167, 1], [1.96299, 2], [0.67198, 2]]
# Numero atomico del litio
Z = 3


In [12]:
f1=STO(zetas[0][0], zetas[0][1])
f2=STO(zetas[1][0], zetas[1][1])
f3=STO(zetas[2][0], zetas[2][1])
f4=STO(zetas[3][0], zetas[3][1])
fs = [f1, f2, f3, f4]

R = np.zeros((len(zetas), len(zetas), len(zetas), len(zetas)))
H = H_matrix(fs, Z)
S = S_matrix(fs)
e, Co = secular_eqn(H, S)
P = P_matrix(Co)
scf_H = get_E0(e, P, H)

##############################################

print('-'*30, "Aproximacion cero", '-'*30)        
display(Math(' energia = {1} \ eV'.format(format(scf_H, '0.5f'), format(scf_H*27.211, '0.5f')))) 

##############################################

for i in range(5):
    print('-'*30, "Aproximacion", i + 1, '-'*30)
    if(i==0):
        start = time.time()
        R = R_matrix(zetas)
    else:
        start = time.time()
    F = F_matrix(fs, Z, zetas, Co, R)
    S = S_matrix(fs)
    e, Co = secular_eqn(F, S)
    P = P_matrix(Co)
    scf_H = get_E0(e, P, H)
    ##########################################        
    display(Math(' energia = {1} \ eV'.format(format(scf_H, '0.5f'), format(scf_H*27.211, '0.5f'))))  

------------------------------ Aproximacion cero ------------------------------


<IPython.core.display.Math object>

------------------------------ Aproximacion 1 ------------------------------


<IPython.core.display.Math object>

------------------------------ Aproximacion 2 ------------------------------


...

<IPython.core.display.Math object>

------------------------------ Aproximacion 4 ------------------------------


<IPython.core.display.Math object>

------------------------------ Aproximacion 5 ------------------------------


<IPython.core.display.Math object>